In [1]:
import tensorflow as tf


2023-03-11 14:57:28.254776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-11 14:57:28.255218: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
source = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'
model_dir = tf.keras.utils.get_file(fname='ssd_mobilenet_v2_320x320_coco17_tpu-8',origin = source, untar=True)
print(model_dir)

/home/suhas/.keras/datasets/ssd_mobilenet_v2_320x320_coco17_tpu-8


In [3]:
model = tf.saved_model.load(model_dir+'/saved_model',tags = ['serve'])

2023-03-11 14:57:43.168924: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-11 14:57:43.170353: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-11 14:57:43.170893: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Magnus): /proc/driver/nvidia/version does not exist
2023-03-11 14:57:43.182423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
signature = model.signatures['serving_default']

In [5]:
import numpy as np
import cv2
# Load an image from a file
image = cv2.imread('Screenshot 2023-03-10 190018.jpg')

# Convert the image from BGR to RGB format
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Resize the image to 320x320 pixels
image = cv2.resize(image, (320, 320))

# Convert the image to a float32 tensor
image = tf.convert_to_tensor(image, dtype=tf.uint8)

# Add a batch dimension to the tensor
image = tf.expand_dims(image, axis=0)

output_tensor = signature(image)


In [6]:
print(output_tensor.keys())

dict_keys(['detection_classes', 'num_detections', 'raw_detection_boxes', 'detection_scores', 'detection_multiclass_scores', 'detection_anchor_indices', 'detection_boxes', 'raw_detection_scores'])


In [14]:
print(output_tensor['detection_boxes'][0])

tf.Tensor(
[[0.5498422  0.51488787 0.7297781  0.7427593 ]
 [0.3159811  0.24362007 0.830695   0.86361945]
 [0.32443208 0.20782459 0.7398998  0.5940603 ]
 [0.29114962 0.22371548 0.6894703  0.7429621 ]
 [0.5498422  0.51488787 0.7297781  0.7427593 ]
 [0.36915946 0.43463087 0.88143134 0.87863183]
 [0.606972   0.2555985  0.930426   0.54398656]
 [0.65117806 0.25002554 0.9420807  0.52729464]
 [0.30890626 0.3202736  0.48692423 0.66410303]
 [0.30313703 0.25841793 0.57485783 0.7940881 ]
 [0.65117806 0.25002554 0.9420807  0.52729464]
 [0.11325721 0.3847238  0.18732291 0.48560676]
 [0.3787956  0.2468631  0.8270074  0.6560172 ]
 [0.61532634 0.52294534 0.80288047 0.8036359 ]
 [0.11325721 0.3847238  0.18732291 0.48560676]
 [0.6627954  0.2899154  0.9344318  0.5116409 ]
 [0.11366232 0.28021494 0.19119234 0.38080242]
 [0.4300896  0.14462665 0.9116676  0.6295259 ]
 [0.38302863 0.2602626  0.8847884  0.8706955 ]
 [0.24657986 0.09897757 0.8822758  0.9461874 ]
 [0.4301023  0.40190268 0.57860535 0.6388266 ]
 [

In [20]:
colors = np.array([[1.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
image = cv2.imread('Screenshot 2023-03-10 190018.jpg')

# Convert the image from BGR to RGB format
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Resize the image to 320x320 pixels
image = cv2.resize(image, (320, 320))

# Convert the image to a float32 tensor
image = tf.convert_to_tensor(image, dtype=tf.float32)

# Add a batch dimension to the tensor
image = tf.expand_dims(image, axis=0)
image_with_boxes = tf.image.draw_bounding_boxes(image,output_tensor['detection_boxes'],colors)

In [21]:
tf.keras.preprocessing.image.array_to_img(image_with_boxes[0]).show()

[12427:12427:0311/151020.959662:ERROR:browser_dm_token_storage_linux.cc(100)] Error: /etc/machine-id contains 0 characters (32 were expected).
[12427:12453:0311/151021.072231:ERROR:bus.cc(399)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[12427:12459:0311/151021.822990:ERROR:bus.cc(399)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[12427:12459:0311/151021.823203:ERROR:bus.cc(399)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[12427:12453:0311/151021.890480:ERROR:bus.cc(399)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[12427:12453:0311/151021.891554:ERROR:bus.cc(399)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on 

Opening in existing browser session.
